# A Tutorial on Formulating and Using QUBO Models
http://meta-analytics.net/references/QUBO%20Tutorial%20%20Version%201-4.pdf

In [1]:
from __future__ import annotations
from dwave_qbsolv import QBSolv
from typing import Set, Tuple, Container, Dict, Sequence
from dataclasses import dataclass
import networkx as nx

In [2]:
def get_q_dict(q: np.ndarray) -> Dict[Tuple[int, int], float]:
    """Returns q matrix in the form of dict from np.ndarray
    
    The dict can be directly passed to dwave_qbsolv.QBSolv
    """
    assert len(q.shape) == 2 and  q.shape[0] == q.shape[1]
    return dict(zip(zip(*np.nonzero(q)), q[q!=0]))

In [3]:
def test_get_q_dict():
    q = np.array([[1,2],
                  [2,4]])
    assert get_q_dict(q) == {(0,0):1, (0,1):2, (1,0):2, (1,1):4}
test_get_q_dict()

In [102]:
def qbsolv_from_ndarray(q: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Returns lowest energy samples from dwave qbsolv.(for test util)"""
    res = QBSolv().sample_qubo(get_q_dict(q)).record
    i = res.energy == res.energy.min()
    return res.sample[i], res.energy[i]

#  The Number Partitioning Problem

$$q_{ii} = s_{i}(s_{i}-c) \\
q_{ij} = q_{ji} = s_{i}s_{j} \\
c = \sum_i{s_{i}}$$

In [103]:
def qubo_number_partition(s: Container[float]) -> np.ndarray:
    """Return Q matrix of number partitioning problem"""
    s = np.array(list(s))
    c = s.sum()
    q = np.outer(s, s)
    np.fill_diagonal(q, s*(s-c))
    return q

In [107]:
def test_qubo_number_partition():
    s = np.array([3,1,1,2,2,1])
    sample, _ = qbsolv_from_ndarray(qubo_number_partition(s))
    assert s[sample[0] == 1].sum() == 5
    assert s[sample[0] == 0].sum() == 5
test_qubo_number_partition()

# The Max-Cut Problem

$$ Minimize\ y = \sum_{(i,j)\in E}(2x_{i}x_{j}-x_{i}-x_{j})$$

In [108]:
@dataclass
class Graph:
    edges: Collection[Collection[int]] # edge is represented as (i, j)
    n_nodes: int
    init_value: Mapping[int, int] = None # init_value[i] indicates init value of node i
    nodes: Sequence[float] = None # value of nodes. len(nodes) must be n_nodes
     
    def __post_init__(self):
        if self.nodes is not None: assert len(self.nodes) == self.n_nodes 
        if self.edges is None: return
        if len(self.edges) == 0: 
            self.edges = None
            return
        self.edges = np.array(list(self.edges))
        assert self.edges.max() < self.n_nodes
        assert self.edges.shape[1] == 2
        if self.init_value is not None:
            assert all(np.array(list(self.init_value)) < self.n_nodes)
        
    @classmethod
    def from_networkx(cls, graph: nx.Graph) -> Graph:
        return cls(edges = graph.edges, n_nodes = graph.number_of_nodes())
    
    def to_networkx_graph(self) -> nx.Graph:
        nxg = nx.Graph()
        nxg.add_nodes_from(range(self.n_nodes))
        nxg.add_edges_from(self.edges)
        return nxg

In [110]:
def test_graph():
    edges = {(1,2), (3,4)}
    graph = Graph(edges, 5)
    assert graph.edges.shape == (2,2)
    nxg = graph.to_networkx_graph()
    _graph = Graph.from_networkx(nxg)
    assert np.array_equal(graph.edges, _graph.edges)
    assert graph.n_nodes == _graph.n_nodes
test_graph()

In [111]:
def qubo_max_cut(g: Graph) -> np.ndarray:
    n_nodes = g.n_nodes
    q = np.zeros((n_nodes, n_nodes))
    i, j = g.edges.T
    np.add.at(q, (i,i), -1)
    np.add.at(q, (j,j), -1)
    q[i,j] += 1
    q[j,i] += 1
    return q

In [112]:
def is_contain(x: np.ndarray, y: np.ndarray) -> bool:
    """If y \in x"""
    assert len(x.shape) -1 == len(y.shape)
    return any(np.array_equal(xi, y) for xi in x)

In [114]:
def test_qubo_max_cut():
    edges = [(0,1), (0,2), (1,3), (2,3), (2, 4), (3, 4)]
    g = Graph(edges=edges, n_nodes=5)
    q = qubo_max_cut(g)
    assert q.tolist() == [[-2,1,1,0,0],
                          [1,-2,0,1,0],
                          [1,0,-3,1,1],
                          [0,1,1,-3,1],
                          [0,0,1,1,-2]]
    sample, _ = qbsolv_from_ndarray(q)
    assert is_contain(sample, np.array([0,1,1,0,0]))
test_qubo_max_cut()

# known penalties

![](img/known_penalties.png)

# The Minimum Vertex Cover Problem (MVC)

$$ y = \sum_{j \in V}x_{j} + P\left(\sum_{(i,j) \in E}\left(1-x_{i}-x_{j}+x_{i}x_{j}\right)\right)$$

In [115]:
def qubo_mvc(g: Graph, p: float = 8.):
    """The Minimum Vertex Cover Problem (MVC)"""
    q = np.diagflat(np.ones(g.n_nodes))
    i, j = g.edges.T
    np.add.at(q, (i,i), -p)
    np.add.at(q, (j,j), -p)
    q[i,j] += p/2.
    q[j,i] += p/2.
    return q

In [117]:
def test_qubo_mvc():
    edges = [(0,1), (0,2), (1,3), (2,3), (2, 4), (3, 4)]
    g = Graph(edges=edges, n_nodes=5)
    q = qubo_mvc(g, p=8.)
    sample,_ = qbsolv_from_ndarray(q)
    assert is_contain(sample, np.array([0,1,1,0,1]))
test_qubo_mvc()

# The Weighted Minimum Vertex Cover Problem (W-MVC)

$$ y = \sum_{j \in V}w_{j}x_{j} + P\left(\sum_{(i,j) \in E}\left(1-x_{i}-x_{j}+x_{i}x_{j}\right)\right)$$

In [118]:
def qubo_wmvc(g: Graph, p: float = 8.):
    """The Minimum Vertex Cover Problem (MVC)"""
    q = np.diagflat(np.ones(g.n_nodes)*g.nodes)
    i, j = g.edges.T
    np.add.at(q, (i,i), -p)
    np.add.at(q, (j,j), -p)
    q[i,j] += p/2.
    q[j,i] += p/2.
    return q

In [119]:
def test_qubo_wmvc():
    edges = [(0,1), (0,2), (1,3), (2,3), (2, 4), (3, 4)]
    g = Graph(edges=edges, n_nodes=5, nodes=np.ones(5))
    q = qubo_mvc(g, p=8.)
    sample, _ = qbsolv_from_ndarray(q)
    assert is_contain(sample, np.array([0,1,1,0,1]))
test_qubo_wmvc()

## Remarks about the scalar penalty P

> Generally, there is a ‘Goldilocks region’ of considerable size that contains
penalty values that work well. A little preliminary thought about the model can yield a ballpark
estimate of the original objective function value. Taking P to be some percentage (75% to
150%) of this estimate is often a good place to start. 

# The Set Packing Problem
$$min\ \sum_{j=1}^n w_{j}x_{j} \\
st \\
\sum_{j=1}^n a_{ij}x_{j} \le 1
$$

In [120]:
from itertools import combinations

In [121]:
def qubo_set_pack(a: np.ndarray, weight: np.ndarray, p=8.) -> np.ndarray:
    assert len(a.shape) == 2
    assert a.shape[-1] == len(weight)
    q = -np.diagflat(weight)
    c = np.einsum("ij,ik->ijk",a,a).astype(np.float) / 2. # constraint
    c *= (1 - np.eye(a.shape[-1]))[None,...] # zeros diag
    q += p*c.sum(0)
    return q

In [122]:
def test_qubo_set_pack():
    a = np.array([[1,0,1,1],
                  [1,1,0,0]])
    w = np.ones(4)
    q = qubo_set_pack(a, w, p=6.)
    assert q.tolist() == [[-1,3,3,3],
                         [3,-1,0,0],
                         [3,0,-1,3],
                         [3,0,3,-1]]
    assert is_contain(qbsolv_from_ndarray(q)[0], np.array([0,1,1,0]))
test_qubo_set_pack()

# The Max 2-Sat Problem
- 面白いことに，Clauseの数によらないモデルとなる

In [138]:
@dataclass
class Clauses:
    literals: np.ndarray[int]
    signs: np.ndarray[bool]
    def __post_init__(self):
        assert len(self.literals) == len(self.signs)

In [227]:
def qubo_max2sat(c: Clauses):
    n = c.literals.max()+1
    q = np.zeros((n,n))
    i,j=c.literals.T
    si,sj=c.signs.T
    np.add.at(q, (i[sj], i[sj]), ((-1)**si)[sj])
    np.add.at(q, (j[si], j[si]), ((-1)**sj)[si])

    offdiag=np.zeros_like(q)
    np.add.at(offdiag, (i,j), (-1)**(si^sj) / 2.)
    return offdiag + offdiag.T + q

In [234]:
def test_qubo_max2sat():
    l = np.array([[0,1],[0,1],[0,1],[0,1],[0,2],
                  [0,2],[1,2],[1,3],[1,2],[1,2],
                  [2,3],[2,3]])
    s = np.array([[True,True],[True,False],[False,True],[False,False],
                  [False,True],[False,False],[True,False],[True,True],
                  [False,True],[False,False],[True,True],[False,False]])
    q = qubo_max2sat(Clauses(l,s))
    assert q.tolist() == [[1,0,0,0],
                          [0,0,-0.5,0.5],
                          [0,-0.5,0,1],
                          [0,0.5,1,-2]]
    sample, _ = qbsolv_from_ndarray(q)
    assert is_contain(sample, np.array([0,0,0,1]))
test_qubo_max2sat()

# The Set Partitioning Problem (SPP)

In [308]:
def qubo_set_partitioning(c: np.ndarray, a: np.ndarray, p=10.):
    assert c.shape[0] == a.shape[1]
    b = np.ones(a.shape[0])
    return np.diagflat(c - p*2*b.dot(a))  + p*np.einsum("ij,ik->jk",a,a)

In [324]:
def test_qubo_set_partitioning():
    c = np.array([3,2,1,1,3,2])
    a = np.array([[1,0,1,0,0,1],
                  [0,1,1,0,1,1],
                  [0,0,1,1,1,0],
                  [1,1,0,1,0,1]])

    q=qubo_set_partitioning(c, a)
    assert q.tolist() == [[-17,10,10,10,0,20],
                        [10,-18,10,10,10,20],
                        [10,10,-29,10,20,20],
                        [10,10,10,-19,10,10],
                        [0,10,20,10,-17,10],
                        [20,20,20,10,10,-28]]
    sample, _ = qbsolv_from_ndarray(q)
    assert is_contain(sample, np.array([1,0,0,0,1,0]))
test_qubo_set_partitioning()

> The set partitioning problem may be viewed as a form of clustering problem and is
elaborated further in Section 6.

# Graph Coloring

> frequency assignment problems to printed circuit board design problems, 

In [424]:
def qubo_graph_coloring(g: Graph, n_color, p=4) -> np.ndarray:
    n = g.n_nodes
    q=np.zeros((n,n_color,n,n_color))
    
    # each nodes are colored by one color
    i=range(n)
    q[i, :, i, :] = p*(np.ones((n_color,n_color)) - 2*np.eye(n_color))
    
    # adjacent nodes are not colored with same color
    i, j = g.edges.T[...,None]
    k=range(n_color)
    q[i,k,j,k]+=p/2.
    q[j,k,i,k]+=p/2.
    
    return q.reshape(n*n_color,n*n_color)

In [425]:
def test_qubo_graph_coloring():
    edges = [[0,1],[0,4],[1,2],[1,3],[1,4],[2,3],[3,4]]
    g = Graph(edges=edges, n_nodes=5)
    ans=np.zeros(n*c)
    ans[[1,3,8,10,14]]=1
    q=qubo_graph_coloring(g, c)
    assert is_contain(qbsolv_from_ndarray(q)[0],ans)
test_qubo_graph_coloring()

> This approach to coloring problems has proven to be very effective for a wide variety of coloring instances with hundreds of nodes.
らしいが，n_color色に対してn_color qbitを用意しているが，log_2(n_color)ですむような構成法はないかな？